![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DeBertaForTokenClassification.ipynb)

## Import ONNX DeBertaForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DeBertaForTokenClassification` is only available since in `Spark NLP 5.1.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DeBerta models trained/fine-tuned for token classification via `DeBertaForTokenClassification` or `TFDeBertaForTokenClassification`. These models are usually under `Token Classification` category and have `bert` in their labels
- Reference: [TFDeBertaForTokenClassification](https://huggingface.co/docs/transformers/model_doc/deberta#transformers.TFDebertaForTokenClassification)
- Some [example models](https://huggingface.co/models?filter=deberta&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases
- Albert uses SentencePiece, so we will have to install that as well

In [1]:
!pip install -q --upgrade transformers[onnx]==4.29.1 optimum tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.7/455.7 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [davanstrien/deberta-v3-base_fine_tuned_food_ner](https://huggingface.co/davanstrien/deberta-v3-base_fine_tuned_food_ner) model from HuggingFace as an example
- In addition to `TFDeBertaForTokenClassification` we also need to save the `DeBertaTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [1]:
from optimum.onnxruntime import ORTModelForTokenClassification
import tensorflow as tf

MODEL_NAME = 'davanstrien/deberta-v3-base_fine_tuned_food_ner'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

Framework not specified. Using pt to export to ONNX.


model.safetensors:   0%|          | 0.00/735M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu121
/usr/local/lib/python3.10/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:561: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(mid - 1).type_as(relative_pos),
/usr/local/lib/python3.10/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:565: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.ceil(

Let's have a look inside these two directories and see what we are dealing with:

In [2]:
!ls -l {ONNX_MODEL}

total 729924
-rw-r--r-- 1 root root        23 Jan  4 17:56 added_tokens.json
-rw-r--r-- 1 root root      2423 Jan  4 17:56 config.json
-rw-r--r-- 1 root root 736301299 Jan  4 17:56 model.onnx
-rw-r--r-- 1 root root       173 Jan  4 17:56 special_tokens_map.json
-rw-r--r-- 1 root root   2464616 Jan  4 17:56 spm.model
-rw-r--r-- 1 root root       412 Jan  4 17:56 tokenizer_config.json
-rw-r--r-- 1 root root   8648791 Jan  4 17:56 tokenizer.json


In [3]:
!mkdir {ONNX_MODEL}/assets

- As you can see, we need to move `spm.model` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [4]:
# get label2id dictionary
labels = ort_model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(ONNX_MODEL +'/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [5]:
!mv {ONNX_MODEL}/spm.model {ONNX_MODEL}/assets

Voila! We have our `spm.model` and `labels.txt` inside assets directory

In [6]:
!ls -lR {ONNX_MODEL}

onnx_models/davanstrien/deberta-v3-base_fine_tuned_food_ner:
total 727520
-rw-r--r-- 1 root root        23 Jan  4 17:56 added_tokens.json
drwxr-xr-x 2 root root      4096 Jan  4 17:56 assets
-rw-r--r-- 1 root root      2423 Jan  4 17:56 config.json
-rw-r--r-- 1 root root 736301299 Jan  4 17:56 model.onnx
-rw-r--r-- 1 root root       173 Jan  4 17:56 special_tokens_map.json
-rw-r--r-- 1 root root       412 Jan  4 17:56 tokenizer_config.json
-rw-r--r-- 1 root root   8648791 Jan  4 17:56 tokenizer.json

onnx_models/davanstrien/deberta-v3-base_fine_tuned_food_ner/assets:
total 2412
-rw-r--r-- 1 root root     331 Jan  4 17:56 labels.txt
-rw-r--r-- 1 root root 2464616 Jan  4 17:56 spm.model


## Import and Save DeBertaForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [7]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.2.2
setup Colab for PySpark 3.2.3 and Spark NLP 5.2.2


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `DeBertaForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DeBertaForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [9]:
from sparknlp.annotator import *
from sparknlp.base import *

tokenClassifier = DeBertaForTokenClassification.loadSavedModel(
     ONNX_MODEL,
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your DeBertaForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 721576
-rw-r--r-- 1 root root 736413822 Jan  4 17:57 deberta_classification_onnx
-rw-r--r-- 1 root root   2464616 Jan  4 17:57 deberta_spp
drwxr-xr-x 3 root root      4096 Jan  4 17:57 fields
drwxr-xr-x 2 root root      4096 Jan  4 17:57 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DeBertaForTokenClassification model 😊

In [13]:
tokenClassifier_loaded = DeBertaForTokenClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [14]:
# .getClasses was introduced in spark-nlp==3.4.0
tokenClassifier_loaded.getClasses()

['I-UNIT',
 'U-PHYSICAL_QUALITY',
 'L-COLOR',
 'U-UNIT',
 'U-COLOR',
 'I-QUANTITY',
 'B-FOOD',
 'U-PART',
 'U-TASTE',
 'U-PROCESS',
 'L-PURPOSE',
 'I-PART',
 'I-PURPOSE',
 'B-PURPOSE',
 'L-PROCESS',
 'L-QUANTITY',
 'B-UNIT',
 'B-QUANTITY',
 'U-FOOD',
 'B-PART',
 'I-TASTE',
 'B-TASTE',
 'L-TASTE',
 'L-FOOD',
 'I-PROCESS',
 'L-PHYSICAL_QUALITY',
 'B-PHYSICAL_QUALITY',
 'I-PHYSICAL_QUALITY',
 'I-FOOD',
 'B-PROCESS',
 'B-COLOR',
 'O',
 'L-PART',
 'L-UNIT',
 'U-QUANTITY']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["I love spicy food"]]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "ner.result").show()

+-----------------+--------------------+
|             text|              result|
+-----------------+--------------------+
|I love spicy food|[O, O, U-TASTE, L...|
+-----------------+--------------------+



That's it! You can now go wild and use hundreds of `DeBertaForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
